<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/Create_testset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Create testset**

Control全体のリストから、train&valで用いたリストを引き算して、
残りからランダムにcontrolのテストセットを作成する

In [1]:
import torch
from IPython.display import Image, clear_output
import os
import shutil
import glob

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#GDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Setup complete. Using torch 1.12.1+cu113 CPU
Mounted at /content/drive


In [74]:
cont_parent_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Control_photo_1618mai"
cont_train_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/gravcont_250px/cont"

cont_parent_list = [i for i in os.listdir(cont_parent_path)]
cont_train_list = [i for i in os.listdir(cont_train_path)]

In [75]:
#parentからtrainを引き算
cont_test_list = set(cont_parent_list) - set(cont_train_list)

In [76]:
import random
random.seed(0) 

#ランダムに46画像を選択
cont_test_list = random.sample(cont_test_list, 46)
cont_test_list = sorted(cont_test_list)

#絶対パスに戻す
cont_test_path = [f"{cont_parent_path}/{basename}" for basename in cont_test_list]


###**Image2square**

In [78]:
import time
from PIL import Image 
def convert(in_list, out_path):
    #処理時間の計測
    start = time.time()

    l=0
    for i in in_list:      
          img = Image.open(i)
          img_new = expand2square(img, (0, 0, 0)).resize((250, 250))
          img_new.save(out_path +'/'+ os.path.basename(i))
          print(out_path +'/'+ os.path.basename(i))

    print('Process done!!')
    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width-height)//2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, (0, (height - width) // 2))
        return result

In [ ]:
# Output
dst_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px"

if os.path.exists(dst_path):
    shutil.rmtree(dst_path)
os.makedirs(dst_path)

convert(cont_test_path, dst_path)

# **Cont_parentのpathを簡略化 (固有idから患者idのみに変換）**

/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/9765-20191106-4-104153_6d5ef3fd8e520567986e16cd3b8c26d905aefcb84155a7af10124d851cdcc2c9.jpg

→

/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/9765.jpg

※ リネームしたら前半の抽出 → img2squareをやり直す


In [ ]:
#cont_parentのパスをリスト化する
cont_parent_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Control_photo_1618mai"
os.listdir(cont_parent_path)

cont_parent_list = glob.glob(f"{cont_parent_path}/*")
cont_parent_list

In [70]:
#パス名にハイフンが含まれていたら、ハイフンより右側を削除する
for orig_path in cont_parent_list:
    if "-" in orig_path:
        dst_path = os.path.dirname(orig_path) + "/" + os.path.basename(orig_path).split("-")[0] + os.path.splitext(os.path.basename(orig_path))[1]
        os.rename(orig_path, dst_path)
        print(f"{orig_path} --> {dst_path}")
    else:
        pass

In [72]:
#拡張子が重複しているものを修正する
for orig_path in cont_parent_list:
    if ".jpg.jpg" in orig_path:
        dst_path = orig_path.rsplit(".",1)[0]
        os.rename(orig_path, dst_path)
        print(f"{orig_path} --> {dst_path}")
    elif ".JPG.JPG" in orig_path:
        dst_path = orig_path.rsplit(".",1)[0]
        os.rename(orig_path, dst_path)
        print(f"{orig_path} --> {dst_path}")
    else:
        pass

#**Contデータセットのフルサイズ版を作成する**

In [ ]:
cont_parent_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Control_photo_1618mai"
cont_children_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px"
dst_children_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_full"

#basename
cont_children_list = os.listdir(cont_children_path)

if os.path.exists(dst_children_path):
    shutil.rmtree(dst_children_path)
os.makedirs(dst_children_path)

for path in cont_children_list:
    shutil.copy(os.path.join(cont_parent_path, path), os.path.join(dst_children_path, path))
    print(os.path.join(dst_children_path, path))

# **Gravフォルダの"スライド"を"slide"に変換する**

/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_250px/スライド6.jpeg 

--> 

/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_250px/slide6.jpeg

（utf-8形式に変換するため）

In [ ]:
grav_parent_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_photo"
grav_test_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_250px"


#拡張子が重複しているものを修正する
for orig_path in glob.glob(f"{grav_test_path}/*"):
    if "スライド" in orig_path:
        dst_path = orig_path.replace("スライド","slide")
        os.rename(orig_path, dst_path)
        print(f"{orig_path} --> {dst_path}")
    else:
        pass